<a href="https://www.kaggle.com/code/probalkar/diabetic-retinopathy-detection?scriptVersionId=171456432" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Loading dependencies

In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

## Importing CSV dataset

In [26]:
data = pd.read_csv("/kaggle/input/diabetic-retinopathy-resized/trainLabels_cropped.csv")
print(data.value_counts().sum())
data.head()

35108


,Unnamed: 0.1,Unnamed: 0,image,level
0,0,0,10_left,0
1,1,1,10_right,0
2,2,2,13_left,0
3,3,3,13_right,0
4,4,4,15_left,1


Adding image names with extension in the dataframe

In [27]:
data['image'] = data['image']+'.jpeg'
data.head()

,Unnamed: 0.1,Unnamed: 0,image,level
0,0,0,10_left.jpeg,0
1,1,1,10_right.jpeg,0
2,2,2,13_left.jpeg,0
3,3,3,13_right.jpeg,0
4,4,4,15_left.jpeg,1


Dropping unnecessary columns from the dataframe

In [28]:
data.drop(columns=['Unnamed: 0.1','Unnamed: 0'], inplace=True)
data.head()

,image,level
0,10_left.jpeg,0
1,10_right.jpeg,0
2,13_left.jpeg,0
3,13_right.jpeg,0
4,15_left.jpeg,1


Randomly selecting half the number of datapoints from the dataset to train and evaluate the model

In [29]:
random_rows = data.sample(n=data.value_counts().sum()//2, random_state=42)
print(random_rows)

                  image  level
13810  17340_right.jpeg      0
22083  27852_right.jpeg      0
27552   34901_left.jpeg      4
15987  20070_right.jpeg      0
16171  20273_right.jpeg      0
...                 ...    ...
17169   21568_left.jpeg      2
18476   23289_left.jpeg      0
21670   27301_left.jpeg      2
13149   16540_left.jpeg      0
1235     1520_left.jpeg      1

[17554 rows x 2 columns]


Splitting the dataset into training and validation set with 20% data for evaluation

In [30]:
train, val = train_test_split(random_rows, test_size=0.2, random_state=42)

## Image preprocessing
Rescaling the images

In [31]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

Declaring constants

In [32]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 5

Creating training and validation set of images

In [33]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory='../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/',
    x_col='image',
    y_col='level',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw')

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val,
    directory='../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/',
    x_col='image',
    y_col='level',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw')

Found 14043 validated image filenames.
Found 3511 validated image filenames.


## MobileNetV2 Pre-trained model
Loading pre-trained model MobileNetV2 with ImageNet classification labels

In [34]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [35]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [36]:
model = Model(inputs=base_model.input, outputs=predictions)

In [37]:
for layer in base_model.layers:
    layer.trainable = False

Compiling the model

In [38]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Training the model

In [39]:
model.fit(train_generator, validation_data=val_generator, epochs=50)

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


439/439 ━━━━━━━━━━━━━━━━━━━━ 137s 288ms/step - accuracy: 0.7258 - loss: 0.9714 - val_accuracy: 0.7411 - val_loss: 0.7734
Epoch 2/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 120s 270ms/step - accuracy: 0.7360 - loss: 0.7934 - val_accuracy: 0.7425 - val_loss: 0.7602
Epoch 3/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 119s 268ms/step - accuracy: 0.7434 - loss: 0.7669 - val_accuracy: 0.7457 - val_loss: 0.7573
Epoch 4/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 121s 272ms/step - accuracy: 0.7392 - loss: 0.7600 - val_accuracy: 0.7474 - val_loss: 0.7544
Epoch 5/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 119s 267ms/step - accuracy: 0.7489 - loss: 0.7418 - val_accuracy: 0.7511 - val_loss: 0.7564
Epoch 6/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 122s 275ms/step - accuracy: 0.7453 - loss: 0.7374 - val_accuracy: 0.7511 - val_loss: 0.7483
Epoch 7/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 121s 272ms/step - accuracy: 0.7635 - loss: 0.7015 - val_accuracy: 0.7499 - val_loss: 0.7533
Epoch 8/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 122s 273ms/step - accuracy: 0.7691 - loss: 0.68

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



439/439 ━━━━━━━━━━━━━━━━━━━━ 132s 298ms/step - accuracy: 0.7720 - loss: 0.6490 - val_accuracy: 0.7539 - val_loss: 0.7522
Epoch 13/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 131s 295ms/step - accuracy: 0.7793 - loss: 0.6280 - val_accuracy: 0.7522 - val_loss: 0.7977
Epoch 14/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 132s 297ms/step - accuracy: 0.7805 - loss: 0.6215 - val_accuracy: 0.7454 - val_loss: 0.7834
Epoch 15/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 131s 294ms/step - accuracy: 0.7857 - loss: 0.6107 - val_accuracy: 0.7499 - val_loss: 0.7958
Epoch 16/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 132s 297ms/step - accuracy: 0.7876 - loss: 0.5993 - val_accuracy: 0.7405 - val_loss: 0.7941
Epoch 17/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 132s 296ms/step - accuracy: 0.7946 - loss: 0.5780 - val_accuracy: 0.7471 - val_loss: 0.8314
Epoch 18/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 131s 295ms/step - accuracy: 0.7903 - loss: 0.5792 - val_accuracy: 0.7371 - val_loss: 0.8101
Epoch 19/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 129s 291ms/step - accuracy: 0.7999 - los

In [40]:
model.save('/kaggle/working/dbrMbNetV2_50epochs_90.h5')